# Compare commands from V2 versus V1

The V1 (legacy) commands archive is used as a reference standard to validate
the new V2 commands archive.

The primary difference is that V1 non-load commands and load stoppages are 
generated using the Chandra.cmd_states non-load commands table and timelines,
while V2 uses only the Flight Command Events from Google Sheets.

In [1]:
import sys
import os
from pathlib import Path
import difflib
import webbrowser

import numpy as np

# Using dev versions
sys.path.insert(0, '../')
sys.path.insert(0, str(Path.home() / 'git' / 'parse_cm'))
sys.path.insert(0, str(Path.home() / 'git' / 'testr'))

from kadi.commands import get_cmds, conf

In [2]:
from kadi import logger
# If needed for debugging...
# logger.setLevel(1)

### Check data prior to RLTT era (APR1420B)

In this era the commands are copied nearly verbatim, except:
- ``source`` column is added (using ``timeline_id``)
- ``timeline_id`` column is removed
- ``AOSTRCAT`` parameters are included

In [3]:
start = '2002-01-01'
stop = '2020-01-01'

with conf.set_temp('commands_version', '1'):
    cmds1 = get_cmds(start, stop)
with conf.set_temp('commands_version', '2'):
    cmds2 = get_cmds(start, stop)
    cmds2 = cmds2[cmds2['tlmsid'] != 'OBS']

In [4]:
cmds1['date', 'type', 'tlmsid'].write('cmds1.dat', format='ascii.fixed_width', delimiter='|', bookend=False, overwrite=True)
cmds2['date', 'type', 'tlmsid'].write('cmds2.dat', format='ascii.fixed_width', delimiter='|', bookend=False, overwrite=True)

In [5]:
# Diffs should be localized to 2006:295:18:59 (incorrect load stop time
# in v1) and NSM around 2008:225:10:00 (incorrect NSM time in v1)
# 306676,306677d306675
# < 2006:295:18:59:02.748 |      ACISPKT | AA00000000
# < 2006:295:18:59:02.748 |   COMMAND_SW |   AONMMODE
# 306679,306680d306676
# < 2006:295:18:59:03.005 |   COMMAND_SW |   AONM2NPE
# < 2006:295:18:59:07.105 |  MP_TARGQUAT |   AOUPTARQ
# 306683,306685d306678
# < 2006:295:18:59:08.748 |   MP_STARCAT |   AOSTRCAT
# < 2006:295:18:59:12.999 |   COMMAND_HW |      CNOOP
# < 2006:295:18:59:12.999 |   COMMAND_SW |   AOMANUVR
# 306690,306692d306682
# < 2006:295:18:59:14.024 |   COMMAND_SW |   AOACRSTE
# < 2006:295:18:59:14.024 |   COMMAND_SW |    COACTSX
# < 2006:295:18:59:16.074 |   COMMAND_HW |      CNOOP
# 420137d420126
# < 2008:225:10:00:00.000 |   COMMAND_SW |   AONSMSAF
# 420141a420131
# > 2008:225:10:07:13.600 |   COMMAND_SW |   AONSMSAF
!diff cmds1.dat cmds2.dat

306676,306677d306675
< 2006:295:18:59:02.748 |      ACISPKT | AA00000000
< 2006:295:18:59:02.748 |   COMMAND_SW |   AONMMODE
306679,306680d306676
< 2006:295:18:59:03.005 |   COMMAND_SW |   AONM2NPE
< 2006:295:18:59:07.105 |  MP_TARGQUAT |   AOUPTARQ
306683,306685d306678
< 2006:295:18:59:08.748 |   MP_STARCAT |   AOSTRCAT
< 2006:295:18:59:12.999 |   COMMAND_HW |      CNOOP
< 2006:295:18:59:12.999 |   COMMAND_SW |   AOMANUVR
306690,306692d306682
< 2006:295:18:59:14.024 |   COMMAND_SW |   AOACRSTE
< 2006:295:18:59:14.024 |   COMMAND_SW |    COACTSX
< 2006:295:18:59:16.074 |   COMMAND_HW |      CNOOP
420137d420126
< 2008:225:10:00:00.000 |   COMMAND_SW |   AONSMSAF
420141a420131
> 2008:225:10:07:13.600 |   COMMAND_SW |   AONSMSAF


### Generate diffs for commands from 2020-01-01 to 2022-01-01

In this era there are real diffs, but all are acceptable. The diffs have been
reviewed individually by aspect team members.

In [6]:
start = '2020-01-01'
stop = '2022-01-01'

In [7]:
with conf.set_temp('commands_version', '1'):
    cmds1 = get_cmds(start, stop)

In [8]:
with conf.set_temp('commands_version', '2'):
    cmds2 = get_cmds(start, stop)

In [9]:
# Commands V1 does not have starcat parameters or LOAD_EVENT commands
for cmds in cmds1, cmds2:
    ok = cmds['type'] == 'MP_STARCAT'
    cmds['params'][ok] = {}
    ok = cmds['type'] == 'LOAD_EVENT'
    cmds.remove_rows(np.where(ok)[0])
    cmds.sort(['date', 'step', 'scs'])

In [10]:
# There are differences in the number of commands
len(cmds1), len(cmds2)

(156600, 156636)

In [11]:
lines1 = cmds1.pformat_like_backstop(show_source=False, show_nonload_meta=False, max_params_width=40,
                                     sort_orbit_events=True)
lines2 = cmds2.pformat_like_backstop(show_source=False, show_nonload_meta=False, max_params_width=40, 
                                     sort_orbit_events=True)

In [12]:
htmldiffer = difflib.HtmlDiff()
html = htmldiffer.make_file(lines1, lines2, 'kadi commands v1', 'kadi commands v2', context=True)

out = Path('commands_v1_v2_diff.html')
out.write_text(html)

url = 'file://' + str(out.absolute())
webbrowser.open(url)

True